In [ ]:
# This extension reloads external Python files
from pathlib import Path
from collections import defaultdict

import numpy as np
import random
import sys
import pandas as pd

import torch
from torch.utils.data import DataLoader
from torch import nn

# allow imports when running script from within project dir
[sys.path.append(i) for i in ['.', '..']]

# local
from src.model.dino_model import get_dino
from src.model.train import *
from src.model.data import *
from src.helpers.helpers import create_paths

# seed
SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)
np.random.seed(SEED)

DATA_PATH = Path('/','cluster', 'scratch', 'thobauma', 'dl_data')
MAX_PATH = Path('/','cluster', 'scratch', 'mmathys', 'dl_data')

BASE_ADV_PATH = Path(MAX_PATH, 'adversarial_data_tensors')
BASE_POSTHOC_PATH = Path(MAX_PATH, 'posthoc_tensors')
POSTHOC_MODELS_PATH = Path(MAX_PATH, 'posthoc_models')

ORI_PATH = Path(DATA_PATH, 'ori_data')
CLASS_SUBSET_PATH = Path(ORI_PATH, 'class_subset.npy')
CLASS_SUBSET = np.load(CLASS_SUBSET_PATH)

ADV_DATASETS = ['cw', 'fgsm_06', 'pgd_03']

DATASETS = [*ADV_DATASETS, 'ori_data']

In [ ]:
INDEX_SUBSET = None
NUM_WORKERS= 0
PIN_MEMORY=True

BATCH_SIZE = 256
EPOCHS= 3
DEVICE = 'cuda'

In [ ]:
DATA_PATHS = create_paths(data_name='ori',
                 datasets_paths=None,  
                 initial_base_path=DATA_PATH, 
                 posthoc_base_path=BASE_POSTHOC_PATH, 
                 train_str='train', 
                 val_str='validation')
for adv_ds in ADV_DATASETS:
    DATA_PATHS = create_paths(data_name=adv_ds,
                 datasets_paths=DATA_PATHS,  
                 initial_base_path=BASE_ADV_PATH, 
                 posthoc_base_path=BASE_POSTHOC_PATH, 
                 train_str='train', 
                 val_str='validation')

In [ ]:
def prepare_data_df(adv_datasets, dataset_paths):
    train_dfs = {}
    for ds in adv_datasets:
        train_dfs[ds] = pd.read_csv(Path(BASE_POSTHOC_PATH, ds, 'train', 'labels_merged.csv'))

    val_dfs = {}
    for ds in adv_datasets:
        val_dfs[ds] = pd.read_csv(Path(BASE_POSTHOC_PATH, ds, 'validation', 'labels_merged.csv'))

    # get adversarial tuples
    for name, df in train_dfs.items():
        df=df[df['true_labels']==df['ori_pred']]
        df=df[df['true_labels']!=df[name+'_pred']]
        df =df[['file', 'true_labels', 'ori_pred', name+'_pred']]
        train_dfs[name]=df
    return train_dfs, val_dfs

In [ ]:
train_dfs, val_dfs = prepare_data_df(ADV_DATASETS, DATA_PATHS)

In [ ]:
# Linear Binary Classifier
class LinearBC(nn.Module):
    def __init__(self, input_shape):
        self.num_labels = 2
        super(LinearBC,self).__init__()
        self.fc1 = nn.Linear(input_shape,2)

    def forward(self, x):
        x = self.fc1(x)
        return x

In [ ]:
def train_posthoc_classifier(adv_datasets, dataset_paths, epochs=EPOCHS):
    logger_dict = {}
    for ds in adv_datasets:
        ds_p = dataset_paths[ds]['posthoc']
        print("#"*50 + f''' training linear classifier for {ds} ''' + "#"*50)
        # loaders
        ori_train = dataset_paths['ori']['posthoc']['train']['images']
        adv_train = ds_p['train']['images']
        print(f'''original images: {ori_train}''')
        print(f'''adversarial images: {adv_train}''')
        train_set = PosthocTrainDataset(ori_train, adv_train, train_dfs[ds])
        train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY, shuffle=True)
        
        
        ori_validation = dataset_paths['ori']['posthoc']['validation']['images']
        adv_validation = ds_p['validation']['images']
        print(f'''original images: {ori_validation}''')
        print(f'''adversarial images: {adv_validation}''')
        val_set = PosthocTrainDataset(ori_validation, adv_validation, val_dfs[ds])
        val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY, shuffle=False)

        print(f'''train samples: {len(train_set)} ''')
        print(f'''val samples: {len(val_set)} \n''')

        # Initialise network
        classifier = LinearBC(1536)
        criterion = nn.CrossEntropyLoss()
        classifier.cuda()
        optimizer = torch.optim.Adagrad(classifier.parameters(), lr=0.001, lr_decay=1e-08, weight_decay=0)
        logger_dict[ds] = train(model=None, 
                                classifier=classifier, 
                                train_loader=train_loader, 
                                validation_loader=val_loader, 
                                log_dir=Path(POSTHOC_MODELS_PATH,ds),
                                tensor_dir=None, 
                                optimizer=optimizer, 
                                criterion=criterion, 
                                adversarial_attack=None, 
                                epochs=epochs, 
                                val_freq=1, 
                                batch_size=16,  
                                lr=0.001, 
                                to_restore = {"epoch": 0, "best_acc": 0.}, 
                                n=4, 
                                avgpool_patchtokens=False)

        print(f'''\n''')
    return logger_dict

In [ ]:
loggers = train_posthoc_classifier(['cw'], DATA_PATHS, 5)

In [ ]:
class AdvDataset(torch.utils.data.Dataset):
    def __init__(self, or_img_folder, adv_img_folder, index_df):
        super().__init__()
        self.or_img_folder = or_img_folder
        self.adv_img_folder = adv_img_folder
        self.index_df = index_df
    
    def __len__(self):
        return len(self.index_df)*2
    
    def __getitem__(self, index):            
        filename = self.index_df['file'].iloc[index%len(self.index_df)]
        filename = filename.split('.')[0]+'.pt'
        if index >= len(self.index_df):
            payload = torch.load(Path(self.or_img_folder, filename)).cpu()
            label = 0 #torch.tensor(0, dtype=torch.float32)
        else:
            payload = torch.load(Path(self.adv_img_folder, filename)).cpu()
            label = 1 #torch.tensor(1, dtype=torch.float32)
        return payload, label, filename

In [ ]:
log_dir = Path(POSTHOC_MODELS_PATH,'fgsm_06')
classifier = LinearBC(1536)
classifier.cuda()

to_restore={'epoch':3}

utils.restart_from_checkpoint(
    Path(log_dir, "checkpoint.pth.tar"),
    run_variables=to_restore,
    state_dict=classifier
)